In [33]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import time
import geopandas as gdp
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
from getpass import getpass
import re


In [2]:
#token = getpass()

In [34]:
client = MongoClient("localhost:27017")

db = client["ironhack"]


In [35]:
db.list_collection_names()

['nyrest', 'Crunchbase', 'restaurants', 'nyneigh']

In [36]:
c = db.get_collection("Crunchbase")

In [37]:
company_category = set()

In [15]:
#df_tech_general = pd.DataFrame()

In [39]:
#def tech_df_function():

tech_name_regex = {"name":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_category_regex ={"category_code":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_description_regex = {"description":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_tags_regex = {"tag_list":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}

tech_cat_or_descr_or_tags = {"$or": [tech_name_regex, tech_category_regex, tech_description_regex, tech_tags_regex]}

projection_name_category_tags = {"_id" : 0, 
                                    "name" : 1, 
                                    "category_code": 1, 
                                    "tag_list": 1,
                                    "total_money_raised": 1,
                                    "offices.country_code" : 1, 
                                    "offices.city" : 1, 
                                    "offices.zip_code" : 1, 
                                    "offices.latitude" : 1, 
                                    "offices.longitude" : 1}

tech_companies_list = list(c.find(tech_cat_or_descr_or_tags, projection_name_category_tags).sort("offices.city", 1))
df_tech_general = pd.DataFrame(tech_companies_list)

    #return df_tech_general

In [40]:
#tech_df_function()

In [41]:
df_tech_general.shape

(285, 5)

In [42]:
def clean_monetary_values(dataframe, column):
    
    for index, row in dataframe.iterrows():
        if pd.isnull(row[column]):
            continue
        
        match = re.search('(\d+\.\d+|\d+)([MmKk])', row[column])
        if not match:
            dataframe.at[index, column] = pd.np.nan
            continue
        
        value, letter = match.groups()
        value = float(value)
        if value <= 0:
            dataframe.at[index, column] = pd.np.nan
            continue
        
        if letter.upper() == 'M':
            value *= 1000000
        else:
            value *= 1000
        
        dataframe.at[index, column] = value
    
    return dataframe


In [43]:
clean_monetary_values(df_tech_general, "total_money_raised")

C:\Users\germa\AppData\Local\Temp\ipykernel_11844\4024254625.py:9: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  dataframe.at[index, column] = pd.np.nan


,name,category_code,tag_list,total_money_raised,offices
0,Sparter,games_video,"gaming, game, wow, worldofwarcraft, virtualgoods",NaN,"[{'zip_code': None, 'city': None, 'country_cod..."
1,Devunity,web,"techcrunch50, tc50",100000.0,[]
2,TechJuicer,None,None,NaN,[]
3,Lockergnome,None,"tech-news, it-news, blog",NaN,[]
4,Apperceptive,web,design,NaN,[]
...,...,...,...,...,...
280,Pacecode Technologies,consulting,"website-development-in-india, website-design-i...",NaN,"[{'zip_code': '600020', 'city': 'chennai', 'co..."
281,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide",NaN,"[{'zip_code': '110075', 'city': 'new delhi', '..."
282,Tech Support 4 NYC,consulting,"computer-consulting, tech-support, new-york-city",NaN,"[{'zip_code': '10018', 'city': 'new york', 'co..."
283,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",500000.0,"[{'zip_code': '94111', 'city': 'san francisco'..."


In [44]:
#def design_df_function():
   
design_name_regex = {"name":{"$regex":"design", "$options":"i"}}
design_category_regex ={"category_code":{"$regex":"design", "$options":"i"}}
design_description_regex = {"description":{"$regex":"design", "$options":"i"}}
design_tags_regex = {"tag_list":{"$regex":"design", "$options":"i"}}

design_cat_or_descr_or_tags = {"$or": [design_name_regex, design_category_regex, design_description_regex, design_tags_regex]}

design_projection_name_category_tags = {"_id" : 0, 
                                            "name" : 1, 
                                            "category_code": 1, 
                                            "tag_list": 1,
                                            "offices.country_code" : 1, 
                                            "offices.city" : 1, 
                                            "offices.zip_code" : 1, 
                                            "offices.latitude" : 1, 
                                            "offices.longitude" : 1}

design_companies_list = list(c.find(design_cat_or_descr_or_tags, design_projection_name_category_tags).sort("offices.city", 1))
df_design = pd.DataFrame(design_companies_list)
#    return df_design

In [45]:
#design_df_function()

In [46]:
def split_dic_df_columns(dataframe, dataframe_column):
    
    """This funcion takes 2 parameters:
    1. name of given dataframe
    2. name of column that contains dictionaries as values

    It will iterate over the rows, then the specified column, checking for missing values.
    If the values is not missing, then iterate over each 
    dictionary item, and chek if the column to be created already exists.
    Then initialize the column values to zero, and then update values
    with the ones corresponding in the dictionary.
    return the original dataframe updated with a new column per each 
    key in the dictionaries, and its values.
    """
    for i, row in dataframe.iterrows():
        for j in row[dataframe_column]:
            if not pd.isnull(j):
                for x, y in j.items():
                    if x not in dataframe.columns:
                        dataframe[str(x)] = np.nan
                    dataframe.at[i, x] = y
    return dataframe



In [47]:
split_dic_df_columns(df_design, "offices")

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude
0,Jasper Design,None,None,[],NaN,NaN,NaN,NaN,NaN
1,Trunkt,ecommerce,"art, design, directory, marketplace, wholesale",[],NaN,NaN,NaN,NaN,NaN
2,Apperceptive,web,design,[],NaN,NaN,NaN,NaN,NaN
3,Medium Design Group,None,None,[],NaN,NaN,NaN,NaN,NaN
4,BootB,advertising,"online-marketplace, creativity, marketing-serv...",[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
774,ecreative,other,"design, freelance-","[{'zip_code': '80300', 'city': 'istanbul', 'co...",80300,istanbul,TUR,NaN,NaN
775,magento xperts,ecommerce,"magento-company, magento-themes, magento-templ...","[{'zip_code': '700064', 'city': 'kolkata', 'co...",700064,kolkata,IND,NaN,NaN
776,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide","[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
777,25 Pixels Media,other,"web-design, costa-rica, blogs, network, spanis...","[{'zip_code': '', 'city': 'san jose', 'country...",,san jose,CRI,NaN,NaN


In [48]:
split_dic_df_columns(df_tech_general, "offices")

,name,category_code,tag_list,total_money_raised,offices,zip_code,city,country_code,latitude,longitude
0,Sparter,games_video,"gaming, game, wow, worldofwarcraft, virtualgoods",NaN,"[{'zip_code': None, 'city': None, 'country_cod...",NaN,NaN,USA,37.090240,-95.712891
1,Devunity,web,"techcrunch50, tc50",100000.0,[],NaN,NaN,NaN,NaN,NaN
2,TechJuicer,None,None,NaN,[],NaN,NaN,NaN,NaN,NaN
3,Lockergnome,None,"tech-news, it-news, blog",NaN,[],NaN,NaN,NaN,NaN,NaN
4,Apperceptive,web,design,NaN,[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
280,Pacecode Technologies,consulting,"website-development-in-india, website-design-i...",NaN,"[{'zip_code': '600020', 'city': 'chennai', 'co...",600020,chennai,IND,NaN,NaN
281,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide",NaN,"[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
282,Tech Support 4 NYC,consulting,"computer-consulting, tech-support, new-york-city",NaN,"[{'zip_code': '10018', 'city': 'new york', 'co...",10018,new york,USA,40.754142,-73.988360
283,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",500000.0,"[{'zip_code': '94111', 'city': 'san francisco'...",94111,san francisco,USA,37.791576,-122.399363


In [49]:
df_design.sort_values(["city", "zip_code"], axis=0, ascending=True, na_position="first")

,name,category_code,tag_list,offices,zip_code,city,country_code,latitude,longitude
0,Jasper Design,None,None,[],NaN,NaN,NaN,NaN,NaN
1,Trunkt,ecommerce,"art, design, directory, marketplace, wholesale",[],NaN,NaN,NaN,NaN,NaN
2,Apperceptive,web,design,[],NaN,NaN,NaN,NaN,NaN
3,Medium Design Group,None,None,[],NaN,NaN,NaN,NaN,NaN
4,BootB,advertising,"online-marketplace, creativity, marketing-serv...",[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
775,magento xperts,ecommerce,"magento-company, magento-themes, magento-templ...","[{'zip_code': '700064', 'city': 'kolkata', 'co...",700064,kolkata,IND,NaN,NaN
776,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide","[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
777,25 Pixels Media,other,"web-design, costa-rica, blogs, network, spanis...","[{'zip_code': '', 'city': 'san jose', 'country...",,san jose,CRI,NaN,NaN
704,A Beautiful Site,enterprise,"cms, website, websites, web-design, web-develo...","[{'zip_code': '1332', 'city': 'Sidlamafa', 'co...",,seattle,USA,47.620973,-122.347276


In [78]:
df_design["city"].value_counts().head(15)

London           21
New York         17
San Francisco    15
Chicago          13
San Diego        10
Bangalore         8
Los Angeles       8
Chennai           8
San Jose          7
Miami             6
Atlanta           6
Kolkata           5
Palo Alto         5
Houston           5
Toronto           5
Name: city, dtype: int64

In [79]:
df_tech_general["city"].value_counts().head(15)

San Francisco        29
Palo Alto             9
London                9
New York              7
Los Angeles           5
Sunnyvale             4
Cambridge             4
Boston                4
Shanghai              3
San Mateo             3
Gothenburg            3
Vancouver             3
Pacific Palisades     2
Solon                 2
Enschede              2
Name: city, dtype: int64

In [56]:
df_design.to_csv("design_companies.csv")

In [57]:
df_tech_general.to_csv("tech_companies.csv")

In [60]:
df_design.dropna(subset=["city"], inplace=True)

In [61]:
df_tech_general.dropna(subset=["city"], inplace=True )

In [62]:
df_design.shape

(632, 9)

In [63]:
df_tech_general.shape

(225, 10)

In [81]:
df_tech_general.tail()

,name,category_code,tag_list,total_money_raised,offices,zip_code,city,country_code,latitude,longitude
280,Pacecode Technologies,consulting,"website-development-in-india, website-design-i...",NaN,"[{'zip_code': '600020', 'city': 'chennai', 'co...",600020,chennai,IND,NaN,NaN
281,MakeMyLink,advertising,"technology, web-design, seo, software-blog, guide",NaN,"[{'zip_code': '110075', 'city': 'new delhi', '...",110075,new delhi,IND,NaN,NaN
282,Tech Support 4 NYC,consulting,"computer-consulting, tech-support, new-york-city",NaN,"[{'zip_code': '10018', 'city': 'new york', 'co...",10018,new york,USA,40.754142,-73.988360
283,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",500000.0,"[{'zip_code': '94111', 'city': 'san francisco'...",94111,san francisco,USA,37.791576,-122.399363
284,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",500000.0,"[{'zip_code': '94111', 'city': 'san francisco'...",94111,san francisco,USA,37.791576,-122.399363


In [82]:
df_tech_general_1M = df_tech_general[df_tech_general["total_money_raised"] >= 1000000.0]

In [74]:
df_design = df_design[df_design["city"] != ""]

In [75]:
df_design.shape

(613, 9)

In [83]:
df_tech_general_1M.shape

(61, 10)

In [84]:
df_tech_general_1M["city"].value_counts().head(15)

San Francisco    14
London            4
San Mateo         3
Shanghai          3
Austin            2
Palo Alto         2
Santa Barbara     2
Gothenburg        2
New York          2
Seattle           2
Sunnyvale         2
Buenos Aires      2
Boulder           2
Boston            2
Tokyo             2
Name: city, dtype: int64

In [86]:
df_tech_general_1M.columns

Index(['name', 'category_code', 'tag_list', 'total_money_raised', 'offices',
       'zip_code', 'city', 'country_code', 'latitude', 'longitude'],
      dtype='object')

In [87]:
import keplergl

In [89]:


# filter the data to only include companies in San Francisco
design_sf = df_design[df_design['city'] == 'San Francisco']
tech_sf = df_tech_general_1M[df_tech_general_1M['city'] == 'San Francisco']

# combine the data into one dataframe
df_sf = pd.concat([design_sf, tech_sf], ignore_index=True)

# create the Kepler.gl map
map_1 = keplergl.KeplerGl(height=600, data={'sf_companies': df_sf})
map_1


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\germa\miniconda3\envs\ironhack\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'sf_companies':                              name category_code  \
0                          P…